# Deep-MINE Framework

## 1. Importing libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

import json

## 2. Loading data

### 2.1 Reviews

In [ ]:
# load json
reviews = []
with open('/content/drive/MyDrive/Mestrado/Prime_Pantry.json') as f:
    for line in f:
        reviews.append(json.loads(line))

# create dataframe
df_reviews = pd.DataFrame(reviews)
print(df_reviews.shape)
df_reviews.head()

(471614, 12)


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5.0,True,"12 14, 2014",A1NKJW0TNRVS7O,B0000DIWNZ,Tamara M.,Good clinging,Clings well,1418515200,NaN,NaN,NaN
1,4.0,True,"11 20, 2014",A2L6X37E8TFTCC,B0000DIWNZ,Amazon Customer,Fantastic buy and a good plastic wrap. Even t...,Saran could use more Plus to Cling better.,1416441600,NaN,NaN,NaN
2,4.0,True,"10 11, 2014",A2WPR4W6V48121,B0000DIWNZ,noname,ok,Four Stars,1412985600,NaN,NaN,NaN
3,3.0,False,"09 1, 2014",A27EE7X7L29UMU,B0000DIWNZ,ZapNZs,Saran Cling Plus is kind of like most of the C...,"The wrap is fantastic, but the dispensing, cut...",1409529600,4,NaN,NaN
4,4.0,True,"08 10, 2014",A1OWT4YZGB5GV9,B0000DIWNZ,Amy Rogers,This is my go to plastic wrap so there isn't m...,has been doing it's job for years,1407628800,NaN,NaN,NaN


In [ ]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471614 entries, 0 to 471613
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         471614 non-null  float64
 1   verified        471614 non-null  bool   
 2   reviewTime      471614 non-null  object 
 3   reviewerID      471614 non-null  object 
 4   asin            471614 non-null  object 
 5   reviewerName    471578 non-null  object 
 6   reviewText      471233 non-null  object 
 7   summary         471473 non-null  object 
 8   unixReviewTime  471614 non-null  int64  
 9   vote            41716 non-null   object 
 10  image           3568 non-null    object 
 11  style           6933 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 40.0+ MB


In [ ]:
# data cleaning
df_reviews = df_reviews.dropna(subset=['reviewerID', 'asin', 'reviewText'])
df_reviews = df_reviews[['reviewerID', 'asin', 'reviewText']]
print(df_reviews.shape)
df_reviews.head()

(471233, 3)


,reviewerID,asin,reviewText
0,A1NKJW0TNRVS7O,B0000DIWNZ,Good clinging
1,A2L6X37E8TFTCC,B0000DIWNZ,Fantastic buy and a good plastic wrap. Even t...
2,A2WPR4W6V48121,B0000DIWNZ,ok
3,A27EE7X7L29UMU,B0000DIWNZ,Saran Cling Plus is kind of like most of the C...
4,A1OWT4YZGB5GV9,B0000DIWNZ,This is my go to plastic wrap so there isn't m...


In [ ]:
df_reviews['reviewerID'].nunique()

247504

In [ ]:
df_reviews['asin'].nunique()

10814

In [ ]:
df_reviews.shape

(471233, 3)

In [ ]:
df_reviews.isnull().sum()

,0
reviewerID,0
asin,0
reviewText,0


### 2.2 Products

In [ ]:
# load json
products = []
with open('/content/drive/MyDrive/Mestrado/meta_Prime_Pantry.json') as f:
    for line in f:
        products.append(json.loads(line))

# create dataframe
df_products = pd.DataFrame(products)
print(df_products.shape)
df_products.head(2)

(10813, 19)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[],,[Sink your sweet tooth into MILK DUDS Candya d...,,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",,Milk Duds,[],[],[],"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",,,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
# replace empty lists with null values
df_products = df_products.map(lambda x: np.nan if len(x) == 0 else x)
df_products.head(2)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,NaN,NaN,[Sink your sweet tooth into MILK DUDS Candya d...,NaN,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",NaN,Milk Duds,NaN,NaN,NaN,"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",NaN,NaN,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,NaN,NaN,[Sink your sweet tooth into MILK DUDS Candya d...,NaN,"HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween C...","[B019KE37WO, B007NQSWEU]",NaN,Milk Duds,NaN,NaN,NaN,"{'ASIN: ': 'B00005BPJO', 'Item model number:':...","<img src=""https://m.media-amazon.com/images/G/...",NaN,NaN,$5.00,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10813 entries, 0 to 10812
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category         0 non-null      float64
 1   tech1            0 non-null      float64
 2   description      10715 non-null  object 
 3   fit              0 non-null      float64
 4   title            10813 non-null  object 
 5   also_buy         4059 non-null   object 
 6   tech2            0 non-null      float64
 7   brand            10810 non-null  object 
 8   feature          1036 non-null   object 
 9   rank             4876 non-null   object 
 10  also_view        5978 non-null   object 
 11  details          10789 non-null  object 
 12  main_cat         10813 non-null  object 
 13  similar_item     0 non-null      float64
 14  date             0 non-null      float64
 15  price            6750 non-null   object 
 16  asin             10813 non-null  object 
 17  imageURL    

In [ ]:
all_products = df_products['asin'].unique()
all_products.shape

(10812,)

In [ ]:
# data cleaning
df_products = df_products.dropna(subset=['description', 'asin', 'imageURL', 'imageURLHighRes'])
df_products = df_products[['description', 'asin', 'imageURL', 'imageURLHighRes']]
print(df_products.shape)
df_products.head(2)

(8942, 4)


,description,asin,imageURL,imageURLHighRes
0,[Sink your sweet tooth into MILK DUDS Candya d...,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,[Sink your sweet tooth into MILK DUDS Candya d...,B00005BPJO,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
df_products.isnull().sum()

,0
description,0
asin,0
imageURL,0
imageURLHighRes,0


In [ ]:
# get number of unique products
df_products['asin'].nunique()

8941

In [ ]:
# remove the duplicate one
df_products = df_products.drop_duplicates(subset=['asin'])
df_products.shape

(8941, 4)

In [ ]:
missing_products = list(set(all_products) - set(df_products['asin']))
len(missing_products)

1871

In [ ]:
df_reviews['asin'].isin(missing_products).sum()

np.int64(22706)

In [ ]:
df_reviews.merge(df_products, on='asin')

,reviewerID,asin,reviewText,description,imageURL,imageURLHighRes
0,A1NKJW0TNRVS7O,B0000DIWNZ,Good clinging,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,A2L6X37E8TFTCC,B0000DIWNZ,Fantastic buy and a good plastic wrap. Even t...,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,A2WPR4W6V48121,B0000DIWNZ,ok,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,A27EE7X7L29UMU,B0000DIWNZ,Saran Cling Plus is kind of like most of the C...,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,A1OWT4YZGB5GV9,B0000DIWNZ,This is my go to plastic wrap so there isn't m...,[200 sq ft (285 ft x 11-3/4 in x 18.6 m2). Eas...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
...,...,...,...,...,...,...
448462,A2WN3971ONAUJF,B01HI76XS0,I like the taste and convenience of eating the...,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
448463,AE4FU8QRB3KXA,B01HI76XS0,These are delicious and healthy snacks! I wit...,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
448464,A3PWAOKOZ8NNGP,B01HI76XS0,Favorite kind bar flavor,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
448465,A36MOFABIPIPGM,B01HI76XS0,Taste not to be believed. Buy a box for my off...,[These bars are where our journey started and ...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
df_reviews.merge(df_products, on='asin')['reviewerID'].value_counts()

,count
reviewerID,
AMMNGUJK4HQJ5,195
A35Q0RBM3YNQNF,175
AKPG8VQBS0MWR,143
A13J2PGKNMJG1K,143
AXK37UZY8UPYP,139
...,...
A3CZPYMZUVA4RD,1
A3QM69AUBKPFVN,1
A39BG3UYXESFOF,1


In [ ]:
df_reviews.merge(df_products, on='asin')['reviewerID'].nunique()

236741

Matrix -> reviewerID (236,741) x asin (8,941)

Feedbacks: 471,233

## 3. Deep-MINE

In [7]:
# https://discuss.pytorch.org/t/how-to-share-weights-between-two-layers/55541/2
# https://www.kaggle.com/code/ignazio/autoencoder-for-text-in-pytorch


def init_weights():
    pass


class ImageAutoEncoder(nn.Module):

    def __init__(self, n_node=100, kernel_size=3, tie_weights=True):
        super().__init__()

        # encoder
        self.conv1 = nn.Conv2d(3, 64, kernel_size=kernel_size, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=kernel_size, stride=1, padding=1)
        self.fc3 = nn.Linear(25600, n_node, bias=True)

        # decoder
        self.fc4 = nn.Linear(n_node, 25600, bias=True)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=kernel_size, stride=1, padding=1)
        self.conv6 = nn.Conv2d(64, 3, kernel_size=kernel_size, stride=1, padding=1)

        # utils
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.relu = nn.ReLU()  # TODO: change to sigmoid

        # share encoder decoder weight matrices
        if tie_weights:
            self._tie_weights()

    def _tie_weights(self):
        self.fc4.weight.data = self.fc3.weight.data.transpose(0,1)
        self.conv5.weight.data = self.conv2.weight.data.transpose(0,1)
        self.conv6.weight.data = self.conv1.weight.data.transpose(0,1)

    def forward(self, x):
        # encoder
        h = self.relu(self.conv1(x))
        h = self.relu(self.conv2(h))
        h = self.pool(h)
        h = self.fc3(h.reshape(-1, 25600))
        print(h.shape)

        # decoder
        h = self.fc4(h).T
        h = h.reshape(-1, 64, 20, 20)
        h = self.upsample(h)
        h = self.conv5(h)
        x_hat = self.conv6(h)
        return x_hat

    def encode(self, x):
        # encoder
        h = self.relu(self.conv1(x))
        h = self.relu(self.conv2(h))
        h = self.pool(h)
        h = self.fc3(h.reshape(-1, 25600))
        return h

    def decode(self, h):
        # decoder
        h = self.fc4(h).T
        h = h.reshape(-1, 64, 20, 20)
        h = self.upsample(h)
        h = self.conv5(h)
        x_hat = self.conv6(h)
        return x_hat


class TextAutoEncoder(nn.Module):

    def __init__(self, dim_size=32, n_node=100, hidden_size=400, tie_weights=True):
        super().__init__()

        # encoder
        self.fc1 = nn.Linear(dim_size, hidden_size, bias=True)
        self.fc2 = nn.Linear(hidden_size, n_node, bias=True)

        # decoder
        self.fc3 = nn.Linear(n_node, hidden_size, bias=True)
        self.fc4 = nn.Linear(hidden_size, dim_size, bias=True)

        # utils
        self.relu = nn.ReLU()  # TODO: change to sigmoid

        # share encoder decoder weight matrices
        if tie_weights:
            self._tie_weights()

    def _tie_weights(self):
        self.fc3.weight.data = self.fc2.weight.data.transpose(0,1)
        self.fc4.weight.data = self.fc1.weight.data.transpose(0,1)

    def forward(self, x):
        # encoder
        h = self.relu(self.fc1(x))
        h = self.fc2(h)
        print(h.shape)

        # decoder
        h = self.relu(self.fc3(h))
        x_hat = self.fc4(h)
        return x_hat

    def encode(self, x):
        # encoder
        h = self.relu(self.fc1(x))
        h = self.fc2(h)
        return h

    def decode(self, h):
        # decoder
        h = self.relu(self.fc3(h))
        x_hat = self.fc4(h)
        return x_hat


model = TextAutoEncoder()
print(sum(p.numel() for p in model.parameters()))
print(model)

x = torch.randn(1, 32)
print(x.shape)
model(x).shape

106532
TextAutoEncoder(
  (fc1): Linear(in_features=32, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=32, bias=True)
  (relu): ReLU()
)
torch.Size([1, 32])
torch.Size([1, 100])


torch.Size([1, 32])

In [ ]:
model = ImageAutoEncoder()
print(sum(p.numel() for p in model.parameters()))
print(model)

x = torch.randn(1, 3, 40, 40)
print(x.shape)
model(x).shape

5223079
ImageAutoEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc3): Linear(in_features=25600, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=25600, bias=True)
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (upsample): Upsample(scale_factor=2.0, mode='bilinear')
  (relu): ReLU()
)
torch.Size([1, 3, 40, 40])
torch.Size([1, 100])


torch.Size([1, 3, 40, 40])

In [20]:
class DeepMINEModel(nn.Module):

    def __init__(self):
        super().__init__()

        ### parameters ###
        self.n_users  = 200
        self.n_items = 500

        # preference dimensions
        self.dim_u = 20
        self.dim_theta = 12

        # autoencoders dimensions
        self.dim_node = 100

        # regularization hyparameters
        self.lambda_m = 1/40
        self.lambda_d = 1/32
        self.lambda_r = 1/32
        self.lambda_theta = 0.1
        self.lambda_beta = 0.001
        self.lambda_Wfu = 0.001

        # lambda parameters
        self.lambda_w = 20
        self.lambda_b = 0
        self.lambda_q = 20
        self.lambda_c = 0
        self.lambda_n = 20
        self.lambda_t = 0

        ### weights ###
        # user perception factors
        self.u_users = nn.Embedding(self.n_users, self.dim_u)
        self.theta_users = nn.Embedding(self.n_users, self.dim_theta)

        # hidden information factor
        self.v_items = nn.Embedding(self.n_items, self.dim_u)

        # cognition factors
        self.a_1 = nn.Embedding(self.n_users, 1)
        self.a_2 = nn.Embedding(self.n_users, 1)
        self.a_3 = nn.Embedding(self.n_users, 1)

        # embedding layer of information integration
        self.embeddings = nn.Embedding(self.dim_node*3, self.dim_theta)

        # biases
        self.alpha_users = nn.Embedding(self.n_users, 1)
        self.beta_items = nn.Embedding(self.n_items, 1)

        # autoencoders
        self.images_autoencoder = ImageAutoEncoder(n_node=self.dim_node)
        self.descriptions_autoencoder = TextAutoEncoder(n_node=self.dim_node)
        self.reviews_autoencoder = TextAutoEncoder(n_node=self.dim_node)

    def _l2_loss(self, *tensors):
        l2_loss = 0
        for tensor in tensors:
            l2_loss += tensor.pow(2).sum()
        return l2_loss / 2

    def prediction(self,
                   user_id: int,
                   item_id: int,
                   m3,
                   d2,
                   r2,
                   ui_latent_factor,
                   ui_content_factor,
                   vj_hidden_info,
                   ui_bias,
                   ji_bias):
        ### user's cognitive styles ###
        ai1 = self.a_1(user_id)
        ai2 = self.a_2(user_id)
        ai3 = self.a_3(user_id)

        ### information integration ###
        f_c = torch.cat([ai1.T.mm(m3), ai2.T.mm(d2), ai3.T.mm(r2)], dim=-1)
        f_j = f_c.mm(self.embeddings.weight)

        # preference
        x_ij = ui_bias + ji_bias + ui_latent_factor.mm(vj_hidden_info.T) + ui_content_factor.mm(f_j.T)

        return x_ij

    def information_representation(self, image, description, review):
        # latent space
        m3 = self.images_autoencoder.encode(image)
        d2 = self.descriptions_autoencoder.encode(description)
        r2 = self.reviews_autoencoder.encode(review)

        # reconstructions
        image_hat = self.images_autoencoder.decode(m3)
        description_hat = self.descriptions_autoencoder.decode(d2)
        review_hat = self.reviews_autoencoder.decode(r2)

        return {
            'embeddings': [m3, d2, r2],
            'reconstructions': [image_hat, description_hat, review_hat]
        }

    def forward(self, user_id, p_item, p_image, p_description, p_review, n_item, n_image, n_description, n_review):
        ### information representation ###
        p_representations = self.information_representation(p_image, p_description, p_review)
        n_representations = self.information_representation(n_image, n_description, n_review)

        # latent spaces
        p_m3, p_d2, p_r2 = p_representations['embeddings']
        n_m3, n_d2, n_r2 = n_representations['embeddings']

        # reconstructions
        p_image_pred, p_description_pred, p_review_pred = p_representations['reconstructions']
        n_image_pred, n_description_pred, n_review_pred = n_representations['reconstructions']

        ### calculate user preference ###
        ui_latent_factor = self.u_users(user_id)
        ui_content_factor = self.theta_users(user_id)

        # hidden information
        p_vj_hidden_info = self.v_items(p_item)
        n_vj_hidden_info = self.v_items(n_item)

        # biases
        ui_bias = self.alpha_users(user_id)
        p_ji_bias = self.beta_items(p_item)
        n_ji_bias = self.beta_items(n_item)

        ### predictions ###
        x_ij = self.prediction(user_id, p_item, p_m3, p_d2, p_r2, ui_latent_factor, ui_content_factor, p_vj_hidden_info, ui_bias, p_ji_bias)
        x_ik = self.prediction(user_id, n_item, n_m3, n_d2, n_r2, ui_latent_factor, ui_content_factor, n_vj_hidden_info, ui_bias, n_ji_bias)

        ### losses ###
        # information representation
        l1 = (
            (self.lambda_m/2) * torch.nn.functional.mse_loss(p_image_pred, p_image)
            + (self.lambda_w/2) * self._l2_loss(self.images_autoencoder.conv1.weight, self.images_autoencoder.conv2.weight, self.images_autoencoder.fc3.weight)
            + (self.lambda_b/2) * self._l2_loss(self.images_autoencoder.conv1.bias, self.images_autoencoder.conv2.bias, self.images_autoencoder.fc3.bias)
        )
        l2 = (
            (self.lambda_d/2) * torch.nn.functional.mse_loss(p_description_pred, p_description)
            + (self.lambda_q/2) * self._l2_loss(self.descriptions_autoencoder.fc1.weight, self.descriptions_autoencoder.fc2.weight)
            + (self.lambda_c/2) * self._l2_loss(self.descriptions_autoencoder.fc1.bias, self.descriptions_autoencoder.fc2.bias)
        )
        l3 = (
            (self.lambda_r/2) * torch.nn.functional.mse_loss(p_review_pred, p_review)
            + (self.lambda_n/2) * self._l2_loss(self.reviews_autoencoder.fc1.weight, self.reviews_autoencoder.fc2.weight)
            + (self.lambda_t/2) * self._l2_loss(self.reviews_autoencoder.fc1.bias, self.reviews_autoencoder.fc2.bias)
        )

        # preference
        log_likelihood = torch.nn.functional.logsigmoid((x_ij - x_ik).unsqueeze(0)).sum()
        regularization = (
            self._l2_loss(ui_latent_factor) * (self.lambda_theta/2)
            + self._l2_loss(p_vj_hidden_info) * (self.lambda_theta/2)
            + self._l2_loss(ui_content_factor) * (self.lambda_theta/2)
            + self._l2_loss(p_ji_bias) * (self.lambda_beta/2) # B_i?
            + self._l2_loss(self.embeddings.weight) * (self.lambda_Wfu/2)
        )

        loss = -log_likelihood + regularization + l1 + l2 + l3

        return loss

In [ ]:
class DeepMINEDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        # get user's id
        user_id = self.df['reviewerID']

        # get positive implicit feedback item from user
        p_item = self.df['asin']
        p_image = torch.randn(1, 3, 40, 40)
        p_description = torch.randn(1, 32)
        p_review = torch.randn(1, 32)

        # get a random negative feedback item from user
        n_item = self.df['n_item']
        n_image = torch.randn(1, 3, 40, 40)
        n_description = torch.randn(1, 32)
        n_review = torch.randn(1, 32)

        positive_item = {
            'pos_item_id': p_item,
            'pos_item_image': p_image,
            'pos_item_description': p_description,
            'pos_item_review': p_review,
        }

        negative_item = {
            'neg_item_id': n_item,
            'neg_item_image': n_image,
            'neg_item_description': n_description,
            'neg_item_review': n_review,
        }

        return user_id, positive_item, negative_item

In [22]:
# user id
user = torch.LongTensor([4, 34, 1])

# positive item
p_item = torch.LongTensor([50, 9, 5])
p_image = torch.randn(3, 3, 40, 40)
p_description = torch.randn(3, 32)
p_review = torch.randn(3, 32)

# negative item
n_item = torch.LongTensor([98, 78, 3])
n_image = torch.randn(3, 3, 40, 40)
n_description = torch.randn(3, 32)
n_review = torch.randn(3, 32)

model = DeepMINEModel()
model(user, p_item, p_image, p_description, p_review, n_item, n_image, n_description, n_review)

tensor(2066.3901, grad_fn=<AddBackward0>)

In [ ]:
model = DeepMINEModel()
training_data = DeepMINEDataset()
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
optimizer = nn.optim.AdamW(model.parameters(), lr=1e-3)

for epoch in range(100):
    for data in train_dataloader:
        optimizer.zero_grad()
        user_id, positive_item, negative_item = data

        p_item, p_image, p_description, p_review = positive_item['pos_item_id'], positive_item['pos_item_image'], positive_item['pos_item_description'], positive_item['pos_item_review']
        n_item, n_image, n_description, n_review = positive_item['neg_item_id'], positive_item['neg_item_image'], positive_item['neg_item_description'], positive_item['neg_item_review']

        loss = model(user, p_item, p_image, p_description, p_review, n_item, n_image, n_description, n_review)
        loss.backward()
        optimizer.step()

In [ ]:
### parameters ###
n_users  = 200
n_items = 500

# preference dimensions
dim_u = 20
dim_theta = 4

# autoencoders dimensions
dim_node = 100

# regularization hyparameters
lambda_m = 1/40
lambda_d = 1/32
lambda_r = 1/32
lambda_theta = 0.1
lambda_beta = 0.001
lambda_Wfu = 0.001

# lambda parameters
lambda_w = 20
lambda_b = 0
lambda_q = 20
lambda_c = 0
lambda_n = 20
lambda_t = 0

### Weights

In [ ]:
# user perception factors
u_users = nn.Embedding(n_users, dim_u)
theta_users = nn.Embedding(n_users, dim_theta)

# hidden information factor
v_items = nn.Embedding(n_items, dim_u)

# cognition factors
a_1 = nn.Embedding(n_users, 1)
a_2 = nn.Embedding(n_users, 1)
a_3 = nn.Embedding(n_users, 1)

# embedding layer of information integration
embeddings = nn.Embedding(dim_node*3, dim_theta)

# biases
alpha_users = nn.Embedding(n_users, 1)
beta_items = nn.Embedding(n_items, 1)

### Models

In [ ]:
# autoencoders
images_autoencoder = ImageAutoEncoder()
descriptions_autoencoder = TextAutoEncoder()
reviews_autoencoder = TextAutoEncoder()

### Forward

In [ ]:
def prediction(user, item, image, description, review):
    ### information representation ###
    m3 = images_autoencoder.encode(image)
    d2 = descriptions_autoencoder.encode(description)
    r2 = reviews_autoencoder.encode(review)

    ### user's cognitive styles ###
    ai1 = a_1(user)
    ai2 = a_2(user)
    ai3 = a_3(user)

    ### information integration ###
    f_c = torch.cat([ai1.mm(m3), ai2.mm(d2), ai3.mm(r2)], dim=-1)
    f_j = f_c.mm(embeddings.weight)

    ### calculate user preference ###
    ui_latent_factor = u_users(user)
    ui_content_factor = theta_users(user)

    # hidden information
    vj_hidden_info = v_items(item)

    # biases
    ui_bias = alpha_users(user)
    ji_bias = beta_items(item)

    # preference
    x_ij = ui_bias + ji_bias + ui_latent_factor.mm(vj_hidden_info.T) + ui_content_factor.mm(f_j.T)

    return x_ij

In [ ]:
# user id
user = torch.LongTensor([4])

# positive item
p_item = torch.LongTensor([50])
p_image = torch.randn(1, 3, 40, 40)
p_description = torch.randn(1, 32)
p_review = torch.randn(1, 32)

# negative item
n_item = torch.LongTensor([98])
n_image = torch.randn(1, 3, 40, 40)
n_description = torch.randn(1, 32)
n_review = torch.randn(1, 32)

# predictions
x_ij = prediction(user, p_item, p_image, p_description, p_review)
x_ik = prediction(user, n_item, n_image, n_description, n_review)
x_ij - x_ik

tensor([[10.5123]], grad_fn=<SubBackward0>)

In [ ]:
-torch.nn.functional.logsigmoid((x_ij - x_ik).unsqueeze(0)).sum()

tensor(2.7198e-05, grad_fn=<NegBackward0>)

Next steps:
- Regularization parameters to loss function
- Create dataloaders
- Analyze the data
- Evaluation methods

Perhaps check these references:

- https://github.com/sh0416/bpr/blob/master/train.py
- https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/cornac_bpr_deep_dive.ipynb
- https://medium.com/data-science/recommender-system-bayesian-personalized-ranking-from-implicit-feedback-78684bfcddf6
- https://github.com/PreferredAI/cornac/blob/93058c04a15348de60b5190bda90a82dafa9d8b6/cornac/models/vbpr/recom_vbpr.py#L249

**BPR: Bayesian Personalized Ranking**

![](https://seunghan96.github.io/assets/img/recsys/24-4.png)

**VBPR: Visual Bayesian Personalized Ranking**

- Paper: https://arxiv.org/pdf/1510.01784
- Github: https://github.com/aaossa/VBPR-PyTorch
- Github: https://github.com/arogers1/VBPR

![](https://raw.githubusercontent.com/aaossa/VBPR-PyTorch/main/vbpr.png?raw=True)

![](https://velog.velcdn.com/images/hyxxnii/post/732ba898-f3bb-4814-870e-0e49b7e13194/image.png)